## Q1. Running Elastic

**Q:** What's the version.build_hash value?

**A:** build_hash : "42f05b9372a9a4a470db3b52817899b99a76ee73"

In [1]:
!curl localhost:9200

{
  "name" : "9c8c8d4d994a",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "qzciLt5KRhCxDTbQnIfbxw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


## Q2. Indexing the data
**Q:** Which function do you use for adding your data to elastic?

**A:** index


## Q3. Searching


**Q:** What's the score for the top ranking result?

**A:** top score : 84.050095


In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [4]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████| 948/948 [00:24<00:00, 38.50it/s]


In [7]:
response['hits']['hits']


[{'_index': 'course-questions',
  '_id': '8EyYbpABk85mk-ffTVr0',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '_kyYbpABk85mk-ffG1iD',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\n

In [6]:
query = 'How do I execute a command in a running docker container?'

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            }
                }
            }
        }
    


response = es_client.search(index=index_name, body= search_query)

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

**Q:** Return 3 results. What's the 3rd question returned by the search engine?

**A:** How do I copy files from a different folder into docker container’s working directory?

In [9]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body= search_query)


In [14]:
for index, hit in enumerate(response['hits']['hits']):
    print(index + 1, "|" ,hit['_source']['question'])

1 | How do I debug a docker container?
2 | How do I copy files from my local machine to docker container?
3 | How do I copy files from a different folder into docker container’s working directory?
4 | The input device is not a TTY when running docker in interactive mode (Running Docker on Windows in GitBash)
5 | How to fix error after running the Docker run command


## Q5. Building a prompt

In [58]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body= search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [73]:
for hit in search_results:
    print(hit['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


In [63]:
def build_prompt(query, search_results):
    
    context_template = ""
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context_template}
    """.strip()

    # for doc in search_results:
    #     context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    return prompt_template
    

# def llm(prompt):
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": prompt
#             }
#         ],
#         model="llama3-8b-8192",
#     )
    
#     return chat_completion.choices[0].message.content

In [78]:
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)

## Q6. Tokens

**Q:** Use the encode function. How many tokens does our prompt have?

**A:**

In [80]:
import os
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

encoding.encode(str(prompt))

[568,
 63842,
 261,
 4165,
 14029,
 29186,
 13,
 30985,
 290,
 150339,
 4122,
 402,
 290,
 31810,
 8099,
 591,
 290,
 40251,
 7862,
 15043,
 77,
 271,
 7649,
 1606,
 290,
 19719,
 591,
 290,
 31810,
 8099,
 1261,
 55959,
 290,
 150339,
 15043,
 77,
 271,
 2381,
 77,
 271,
 150339,
 25,
 354,
 14921,
 21170,
 77,
 271,
 2381,
 77,
 271,
 31810,
 8099,
 16008,
 77,
 271,
 354,
 3822,
 19739,
 20115,
 461,
 48,
 25,
 354,
 14921,
 21170,
 77,
 271,
 355,
 25,
 354,
 919,
 155700]